# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Introduction: Business Problem <a name="introduction"></a>

Hue and Da Nag are two cities in central Vietnam. They are both on the coast of Vietnam, but Da Nang is more renowned for beaches than Hue. 
Hue on the other hand is known for being the former center of power in Vietnam and was the sett of Royalty until 1947.  
Both a major tourist attractions, or at least were until Covid19. 
Originally I wanted to compare the influence on the cities of the Covid19 lockdown in the fall of 2020 because Da Nang was in lockdown and Hue wasn't. 
As it is not possible to gather data from Foursquare based on date I will compare the two cities as they stand today.
In particular I will look at the tips and see if there is a correlation between the length of the tip and the rating given.


## Data<a name="data"></a>

In [43]:
import pandas as pd
import numpy as np
!pip install geocoder
import geocoder

In [44]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [45]:
address = 'Da Nang, VN'

geolocator = Nominatim(user_agent="da_nang_explorer")
location_danag = geolocator.geocode(address)
latitude_danang = location.latitude
longitude_danang = location.longitude
print('The geograpical coordinate of Da Nang is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Da Nang is 16.4638013, 107.5821911.


In [46]:
address = 'Hue, VN'

geolocator2 = Nominatim(user_agent="hue_explorer")
location = geolocator2.geocode(address)
latitude_hue = location.latitude
longitude_hue = location.longitude
print('The geograpical coordinate of Hue is {}, {}.'.format(latitude_hue, longitude_hue))

The geograpical coordinate of Hue is 16.4638013, 107.5821911.


In [47]:
# create map of Da Nang using latitude and longitude values
map_cities = folium.Map(location=[latitude_danang, longitude_danang], zoom_start=10)

folium.Marker(
    location=[latitude_danang, longitude_danang],
    popup="Da Nang",
    icon=folium.Icon(icon="green"),
).add_to(map_cities)


    
map_cities

## Methodology<a name="methodology"></a>

## Analysis<a name="analysis"></a>

## Results<a name="results"></a>

## Conclusion<a name="conclusion"></a>